<a href="https://colab.research.google.com/github/timmmariya/Empirical-Law-Studies/blob/main/3_%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_(%D1%81%D1%83%D0%B4%2C_%D0%B2%D0%B8%D0%B4%2C_%D0%BA%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F)_%D0%B2_%D1%81%D0%BA%D0%B2%D0%BE%D0%B7%D0%BD%D1%83%D1%8E_%D0%BD%D1%83%D0%BC%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Представленный Colab Notebook является частью серии наработок по исследованиям в области практической оценки судопроизводства методами численного анализа, проводимых [НУЛ анализа данных правоприменения НИУ ВШЭ](https://adp.hse.ru/). 

Контактная информация рабочей группы: gkrasnozhenov@hse.ru (Григорий Григорьевич Красноженов, ЗавЛаб НУЛ АДП), timmariya541@gmail.com (Мария Тимонина, стажер, ответственный за текущую задачу)

Материалом для анализа служат **данные системы арбитражных судов РФ**.

---

#### Стандартный пролог подключения библиотек и загрузки базы данных в pandas DataFrame:

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

plt.style.use('ggplot') # стиль графиков
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_data/APwithEndDates.csv"
df = pd.read_csv(path, sep=',')

# **Постановка задачи**

Объем исходной базы данных достаточно велик. Он составляет порядка 70 ГБ и может быть значимо сокращен посредством замены повторяющейся текстовой информации на систему нумерации с сохранением расшифровки введенных обозначений в формате словарей (индекс-кодировка, замененнная строка).

Объектом оптимизации служат в первую очередь категориальные признаки дел, а именно (см. общий вид исходной базы ниже):
* Суд, в котором рассматривается дела
* Вид спора
* Категория спора

**В этой тетрадке** зафиксированы все шаги планирования и внедрения структуры номерной классификации, организации таблицы расшифровок и налаживания двусторонней конвертации между ними.

Структура исходных данных:

In [ ]:
df.head()

,Номер,Суд,Вид Спора,Категория Спора,Сумма Иска,Сумма Взысканий,Начало,Продолжительность,Конец
0,34925-13/1941,АС Воронежской области,А,О взыскании обязательных платежей и санкций по...,0.0,0.0,2014-01-31,48,2014-03-20
1,А01-120/2014,АС Республики Адыгея,Г,О признании права собственности,0.0,0.0,2014-01-31,8,2014-02-08
2,А01-121/2014,АС Республики Адыгея,Г,О неисполнении или ненадлежащем исполнении обя...,34484.9,36484.9,2014-01-31,670,2015-12-02
3,А01-122/2014,АС СКО,Г,Иные экономические споры,132000.0,176979.0,2014-01-31,419,2015-03-26
4,А01-123/2014,АС Республики Адыгея,А,О привлечении к административной ответственности,0.0,0.0,2014-01-31,42,2014-03-14


---

**!!** Следует отметить, что "сырые" данные КАД содержат **множественные названия некоторых категорий** для споров, вносившихся в базу в разное время или в разных судах. Для корректной обработки данных такие категории необходимо **объединять** в одну. Все подобные случаи задвоения установлены и исправлены на основании экспертизы сотрудников лаборатории наполовину вручную.

**Например,** "О неисполнении или ненадлежащем исполнении обязательств по договорам энергоснабжения" и "Энергоснабжение - Неисполнение или ненадлежащее исполнение обязательств".

# **Реализация**

### Выгрузка текстовой информации для **проведения внутренней экспертизы (аналитической обработки)**

Организация таблицы всех встречающихся в базе пар (вид спора, категория спора) для одного дела с сохранением в формате .xlsx

In [ ]:
c = df[['Категория Спора', 'Вид Спора']].value_counts().sort_values()
c.to_excel('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/output.xlsx')
print('В исходной базе различных категорий:', df['Категория Спора'].value_counts().size)

В исходной базе различных категорий: 431


Резервный (первый) вариант в файл .txt [Вдруг пригодится]:

```
f = open('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/categ_type_sorted.txt', 'w')
for p in c:
    f.write(p[1] + '\t' + p[0] + '\t' + p[2] + '\n')
f.close()
```

* Объединять будем как: 1) одинаковые по сути под одну цифру и хранить в отдельной табличке пары номер - (вид, категория)

In [ ]:
f = open('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/Категоризация/splitation.txt', 'w')
f.write('К1' + '\t' + 'К2' + '\t' + 'К3' + '\t' 'К4' + '\t' +'Расшифровка' + '\n')


filepath = '/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/Категоризация/categs_with_nums.txt'
with open(filepath) as fp:
   line = fp.readline()
   while line:
       c = ['-', '-', '-', '-']
       num = line.split()[0]
       inds = num.split('.')
       for i in range(len(inds) - 1):
          c[i] = inds[i]
       f.write(c[0] + '\t' +
               c[1] + '\t' +
               c[2] + '\t' +
               c[3] + '\t' +
               line[len(num) + 1:] + '\n')
       line = fp.readline()

f.close()
fp.close()

In [ ]:
path = '/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/Категоризация/splitation.txt'
dfs = pd.read_csv(path, sep='\t')

In [ ]:
dfs

,К1,К2,К3,К4,Расшифровка
0,1,-,-,-,Споры о заключении договоров(контрактов)
1,1,1,-,-,Споры о заключении договоров на поставку товар...
2,2,-,-,-,Споры о признании договоров недействительными
3,3,-,-,-,Споры о неисполнении или ненадлежащем исполнен...
4,3,1,-,-,Споры о неисполнении или ненадлежащем исполнен...
...,...,...,...,...,...
241,72,1,3,-,"об оспаривании ненорм. правовых актов органов,..."
242,72,1,4,-,об оспаривании реш-ий федер. антимоноп-ого орг...
243,72,2,-,-,об установлении патентообладателя
244,72,3,-,-,о досрочном прекращении правовой охраны товарн...


In [ ]:
dfs.to_excel('/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/Категоризация/splitation.xlsx')

In [ ]:
dfs.groupby('Расшифровка')['К1'].apply(list)

Расшифровка
Банкротство физического лица                                                         [36]
Дела об оспаривании ненормативных правовых актов Государственной Думы                [54]
Дела об оспаривании ненормативных правовых актов Совета Федерации                    [54]
Иные экономические споры                                                             [34]
Корпоративные споры                                                                  [16]
                                                                                     ... 
обращение Председателя Верховного Суда Российской Федерации                          [60]
обращение граждан и организаций на действие (бездействие) судей                      [60]
по делам о нарушениях, указанных в п. 4 ч. 1 ст. 14 ФЗ "О защите конкуренции"        [31]
споры между высшими органами государственной власти субъектов РФ                     [59]
споры между федеральными органами гос. власти и органами гос. власти субъектов РФ    [58

In [ ]:
dk1 = dfs.groupby('Расшифровка')['К1'].apply(list).to_dict()
dk2 = dfs.groupby('Расшифровка')['К2'].apply(list).to_dict()

In [ ]:
list(dk1.keys())

['Банкротство физического лица',
 'Дела об оспаривании ненормативных правовых актов Государственной Думы',
 'Дела об оспаривании ненормативных правовых актов Совета Федерации',
 'Иные экономические споры',
 'Корпоративные споры',
 'Об обжаловании отказа в государственной регистрации прав на недвижимое имущество и сделок с ним',
 'Об оспаривании зарегистрированных прав на недвижимое имущество и сделок с ним',
 'Об уклонении от государственной регистрации прав на недвижимое имущество и сделок с ним',
 'Об уклонении от государственной регистрации юридических лиц и индивидуальных предпринимателей',
 'Спор, по вопросам вытекающим из деятельности депозитариев',
 'Споры о взыскании обязательных платежей и санкций на основании п. 3 ст. 46 НК РФ',
 'Споры о взыскании обязательных платежей и санкций по налогам и сборам',
 'Споры о взыскании по заявлениям Пенсионного фонда обязательных платежей и санкций с организаций и граждан',
 'Споры о взыскании по заявлениям Фонда социального страхования обя

# **Непосредственная замена**

In [ ]:
df['K1'] = df['Категория Спора'].map(dk1)
df['K2'] = df['Категория Спора'].map(dk2)

In [ ]:
df[df['K1'].notnull()]

,Номер,Суд,Вид Спора,Категория Спора,Сумма Иска,Сумма Взысканий,Начало,Продолжительность,Конец,K1,K2
3,А01-122/2014,АС СКО,Г,Иные экономические споры,132000.0,176979.0,2014-01-31,419,2015-03-26,[34],[-]
7,А01-126/2014,АС СКО,Г,Иные экономические споры,0.0,0.0,2014-01-31,273,2014-10-31,[34],[-]
76,А03-1573/2014,АС Алтайского края,Г,Иные экономические споры,0.0,0.0,2014-01-31,8,2014-02-08,[34],[-]
121,А03-1628/2014,АС Алтайского края,Г,Иные экономические споры,60007.5,0.0,2014-01-31,204,2014-08-23,[34],[-]
131,А03-1643/2014,АС Алтайского края,А,Иные экономические споры,0.0,0.0,2014-01-31,21,2014-02-21,[34],[-]
...,...,...,...,...,...,...,...,...,...,...,...
3627222,А49-6351/2016,АС Пензенской области,Г,Иные экономические споры,20680.1,20680.1,2016-05-25,99,2016-09-01,[34],[-]
3627249,А50-12021/2016,АС Пермского края,Г,Иные экономические споры,0.0,0.0,2016-05-25,41,2016-07-05,[34],[-]
3627291,А50-12077/2016,АС Пермского края,А,"Споры, связанные с применением законодательств...",0.0,0.0,2016-05-25,37,2016-07-01,[28],[-]
3627292,А50-12079/2016,АС Пермского края,Г,"Споры, связанные с применением законодательств...",591678.0,0.0,2016-05-25,62,2016-07-26,[28],[-]


# **Восстановление названий категорий**

In [ ]:
path = "/content/drive/MyDrive/НУЛ АДП Материалы Марии Тимониной/_data/indexed_categories.csv"
df_categs = pd.read_csv(path, sep=',')

In [ ]:
df_categs

,ID,ancestorID,сCode,Descr,typeID
0,1,0,1,Споры о заключении договоров(контрактов),2
1,2,1,1.1,Споры о заключении договоров на поставку товар...,2
2,3,0,2,Споры о признании договоров недействительными,2
3,4,0,3,Споры о неисполнении или ненадлежащем исполнен...,2
4,5,4,3.1,Споры о неисполнении или ненадлежащем исполнен...,2
...,...,...,...,...,...
200,201,200,55.1,Споры по дела об оспаривании ненормативных пра...,8
201,202,200,55.2,Споры по делам об оспаривании ненормативных пр...,8
202,203,0,56,Экономические споры между Российской Федерацие...,8
203,204,0,57,Экономические споры между субъектами Российско...,8
